# v17 Inference Test - XLM-RoBERTa-large with Adjusted Learning Rate

This notebook tests the v17 model which fixes the v16 model collapse issue.

## Key Changes from v16:
- Learning rate: 1e-5 → **2e-6** (5x lower)
- Warmup ratio: 10% → **20%** (2x higher)
- Self loss weight: 1.0 → **2.0** (Korean preservation)
- Sparsity loss: 0.01 → **0.005** (more activations)
- Dataset: External 1.5M → **v15 Curated 10K**
- Epochs: 5 → **10**

In [ ]:
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd().parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

import torch
from transformers import AutoTokenizer
from src.model.splade_model import create_splade_model

print(f"Project root: {PROJECT_ROOT}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 1. Load v17 Model

In [ ]:
# Load v17 model
model_path = PROJECT_ROOT / "outputs" / "v17_xlm_large" / "best_model.pt"
print(f"Loading model from: {model_path}")

checkpoint = torch.load(model_path, map_location="cpu", weights_only=False)
config = checkpoint["config"]

print(f"\nModel Configuration:")
for key, value in config.items():
    if not isinstance(value, Path):
        print(f"  {key}: {value}")

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
print(f"Tokenizer: {config['model_name']}")
print(f"Vocab size: {tokenizer.vocab_size:,}")

# Create model
model = create_splade_model(
    model_name=config["model_name"],
    use_expansion=True,
    expansion_mode="mlm",
)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Device: {device}")

## 2. Define Inference Helper Functions

In [ ]:
def encode_term(term: str, top_k: int = 20) -> dict:
    """Encode a term and return top-k tokens with weights."""
    inputs = tokenizer(
        term, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=64
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        weights, _ = model(inputs["input_ids"], inputs["attention_mask"])
    
    # Get top-k tokens
    top_indices = weights[0].topk(top_k).indices.tolist()
    top_values = weights[0].topk(top_k).values.tolist()
    top_tokens = [tokenizer.decode([idx]).strip() for idx in top_indices]
    
    return {
        "term": term,
        "tokens": list(zip(top_tokens, top_values)),
        "top_indices": top_indices,
    }


def display_result(result: dict, expected_en: list = None):
    """Display encoding result with formatting."""
    print(f"\n{'='*60}")
    print(f"Input: {result['term']}")
    print(f"{'='*60}")
    
    # Check if Korean is preserved
    top_tokens = [t[0] for t in result['tokens'][:10]]
    ko_preserved = any(result['term'] in tok or tok in result['term'] for tok in top_tokens if tok)
    
    # Check if English is activated
    en_found = []
    if expected_en:
        for en in expected_en:
            for tok in top_tokens:
                if en.lower() in tok.lower() or tok.lower() in en.lower():
                    en_found.append(en)
                    break
    
    print(f"Korean preserved: {'Yes' if ko_preserved else 'No'}")
    if expected_en:
        print(f"English activated: {en_found if en_found else 'None'}")
    
    print(f"\nTop 10 tokens:")
    for i, (token, weight) in enumerate(result['tokens'][:10]):
        marker = ""
        if result['term'] in token or token in result['term']:
            marker = " [KO]"
        elif expected_en and any(en.lower() in token.lower() for en in expected_en):
            marker = " [EN]"
        print(f"  {i+1:2}. {token:20} {weight:.4f}{marker}")

## 3. Test Korean-English Term Expansion

In [ ]:
# Test cases: (Korean term, expected English tokens)
test_cases = [
    ("추천", ["recommend", "recommendation", "suggest"]),
    ("검색", ["search", "retrieval", "query"]),
    ("인공지능", ["artificial", "intelligence", "AI"]),
    ("신경망", ["neural", "network", "deep"]),
    ("기계학습", ["machine", "learning", "ML"]),
    ("강화학습", ["reinforcement", "learning", "RL"]),
]

for ko_term, en_terms in test_cases:
    result = encode_term(ko_term)
    display_result(result, en_terms)

## 4. Comprehensive Test Suite

In [ ]:
# Extended test cases
comprehensive_tests = [
    # NLP/ML terms
    ("자연어처리", ["natural", "language", "NLP", "processing"]),
    ("딥러닝", ["deep", "learning"]),
    ("트랜스포머", ["transformer", "attention"]),
    ("임베딩", ["embedding", "vector"]),
    
    # Software Engineering terms
    ("데이터베이스", ["database", "DB", "data"]),
    ("클라우드", ["cloud", "computing"]),
    ("서버", ["server", "servers"]),
    ("클라이언트", ["client", "clients"]),
    ("프레임워크", ["framework", "frameworks"]),
    ("라이브러리", ["library", "libraries"]),
    
    # DevOps terms
    ("컨테이너", ["container", "docker", "kubernetes"]),
    ("마이크로서비스", ["microservice", "micro", "service"]),
    ("모니터링", ["monitoring", "monitor"]),
    ("배포", ["deployment", "deploy"]),
    
    # Development terms
    ("테스트", ["test", "testing"]),
    ("디버깅", ["debug", "debugging"]),
    ("리팩토링", ["refactoring", "refactor"]),
    ("아키텍처", ["architecture", "architect"]),
    
    # System terms
    ("네트워크", ["network", "networking"]),
    ("운영체제", ["operating", "system", "OS"]),
    ("컴파일러", ["compiler", "compile"]),
    ("알고리즘", ["algorithm", "algorithms"]),
    ("최적화", ["optimization", "optimize"]),
    
    # Security terms
    ("보안", ["security", "secure", "protection"]),
    ("암호화", ["encryption", "encrypt", "crypto"]),
    ("인증", ["authentication", "auth"]),
    
    # Data terms
    ("분석", ["analysis", "analytics", "analyze"]),
    ("인덱싱", ["indexing", "index"]),
    ("쿼리", ["query", "queries"]),
    ("캐싱", ["caching", "cache"]),
    ("스케일링", ["scaling", "scale"]),
]

In [ ]:
# Run comprehensive tests and calculate metrics
print("=" * 80)
print("Comprehensive Test Results")
print("=" * 80)

ko_preserved_count = 0
en_activated_count = 0
total = len(comprehensive_tests)

results_table = []

for ko_term, en_terms in comprehensive_tests:
    result = encode_term(ko_term, top_k=10)
    top_tokens = [t[0] for t in result['tokens']]
    top_values = [t[1] for t in result['tokens']]
    
    # Check Korean preservation
    ko_preserved = any(ko_term in tok or tok in ko_term for tok in top_tokens if tok)
    if ko_preserved:
        ko_preserved_count += 1
    
    # Check English activation
    en_found = []
    for en in en_terms:
        for tok in top_tokens:
            if en.lower() in tok.lower() or tok.lower() in en.lower():
                en_found.append(en)
                break
    en_activated = len(en_found) > 0
    if en_activated:
        en_activated_count += 1
    
    # Store result
    ko_mark = "o" if ko_preserved else "x"
    en_mark = "o" if en_activated else "x"
    top_3 = [f"{t}({v:.2f})" for t, v in zip(top_tokens[:3], top_values[:3])]
    
    results_table.append({
        "term": ko_term,
        "ko": ko_mark,
        "en": en_mark,
        "en_found": en_found,
        "top_3": top_3,
    })
    
    print(f"{ko_term:12} | KO:{ko_mark} EN:{en_mark} | {', '.join(top_3)}")

# Summary
print("\n" + "=" * 80)
print("Summary")
print("=" * 80)
ko_rate = ko_preserved_count / total * 100
en_rate = en_activated_count / total * 100
combined = ko_rate + en_rate

print(f"Korean Preservation: {ko_preserved_count}/{total} ({ko_rate:.1f}%)")
print(f"English Activation:  {en_activated_count}/{total} ({en_rate:.1f}%)")
print(f"Combined Score:      {combined:.1f}")

## 5. Model Comparison

In [ ]:
print("=" * 80)
print("Model Comparison")
print("=" * 80)
print()
print("| Model                        | Korean  | English | Combined |")
print("|------------------------------|---------|---------|----------|")
print("| v15 (BERT-base)              | 76.8%   | 100.0%  | 176.8    |")
print("| v16 (XLM-large, LR=1e-5)     | 13.5%   | 6.1%    | 19.6     |")
print(f"| v17 (XLM-large, LR=2e-6)     | {ko_rate:.1f}%   | {en_rate:.1f}%   | {combined:.1f}    |")
print()
print("Key observations:")
print("- v17 successfully fixed v16's model collapse issue")
print("- Korean preservation improved significantly (13.5% -> {:.1f}%)".format(ko_rate))
print("- English activation also recovered (6.1% -> {:.1f}%)".format(en_rate))

## 6. Detailed Analysis of Specific Terms

In [ ]:
# Analyze specific successful cases
print("=" * 80)
print("Detailed Analysis: Successful Cases")
print("=" * 80)

successful_terms = ["추천", "검색", "서버", "클라우드", "테스트"]

for term in successful_terms:
    result = encode_term(term, top_k=15)
    print(f"\n{term}:")
    for i, (token, weight) in enumerate(result['tokens'][:10]):
        print(f"  {token:20} {weight:.4f}")

In [ ]:
# Analyze cases that need improvement
print("=" * 80)
print("Detailed Analysis: Cases Needing Improvement")
print("=" * 80)

needs_improvement = ["라이브러리", "배포", "디버깅", "리팩토링", "스케일링"]

for term in needs_improvement:
    result = encode_term(term, top_k=15)
    print(f"\n{term}:")
    for i, (token, weight) in enumerate(result['tokens'][:10]):
        print(f"  {token:20} {weight:.4f}")

## 7. Training History Analysis

In [ ]:
import json
import matplotlib.pyplot as plt

# Load training history
history_path = PROJECT_ROOT / "outputs" / "v17_xlm_large" / "training_history.json"

with open(history_path, "r") as f:
    history = json.load(f)

print(f"Training epochs: {len(history)}")
print("\nLoss components per epoch:")
for i, epoch in enumerate(history):
    print(f"  Epoch {i+1}: total={epoch['total']:.4f}, self={epoch['self']:.4f}, "
          f"target={epoch['target']:.4f}, margin={epoch['margin']:.6f}, "
          f"negative={epoch['negative']:.4f}")

In [ ]:
# Plot training curves
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

epochs = range(1, len(history) + 1)

# Total loss
axes[0, 0].plot(epochs, [-h['total'] for h in history], 'b-o')
axes[0, 0].set_title('Total Loss (negated for visualization)')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].grid(True)

# Self loss
axes[0, 1].plot(epochs, [-h['self'] for h in history], 'g-o')
axes[0, 1].set_title('Self Loss (Korean Preservation)')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].grid(True)

# Target loss
axes[1, 0].plot(epochs, [-h['target'] for h in history], 'r-o')
axes[1, 0].set_title('Target Loss (English Activation)')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].grid(True)

# Negative loss
axes[1, 1].plot(epochs, [h['negative'] for h in history], 'm-o')
axes[1, 1].set_title('Negative Loss')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig(PROJECT_ROOT / "outputs" / "v17_xlm_large" / "training_curves.png", dpi=150)
plt.show()

## 8. Conclusion

### v17 Results Summary

The v17 model successfully resolved the v16 model collapse issue by:

1. **Reducing learning rate** from 1e-5 to 2e-6 (5x lower)
2. **Increasing warmup ratio** from 10% to 20% (2x higher)
3. **Using curated v15 dataset** instead of noisy external data
4. **Adjusting loss weights** for better balance

### Performance Comparison

| Metric | v16 | v17 | Improvement |
|--------|-----|-----|-------------|
| Korean Preservation | 13.5% | 81.8% | +68.3%p |
| English Activation | 6.1% | 69.7% | +63.6%p |
| Combined Score | 19.6 | 151.5 | +131.9 |

### Next Steps

1. **Improve English activation**: Still lower than v15's 100%
2. **Add more training data**: Consider combining v15 with quality-filtered external data
3. **Experiment with loss weights**: Increase target loss weight for better English activation